In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time
import requests
import gzip
import os

fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))
train_images = train_images / 255.0
test_images = test_images / 225.0

In [2]:
def get_testset():
    url = 'https://courses.engr.illinois.edu/ece498icc/sp2019/lab2_request_dataset.php'
    values = {'request': 'testdata', 'netid':'smeng10', 'team':'msp'}
    r = requests.post(url, data=values, allow_redirects=True)
    filename = r.url.split("/")[-1]
    testset_id = filename.split(".")[0].split("_")[-1]
    with open(filename, 'wb') as f: 
        f.write(r.content)
    return load_dataset(filename), testset_id

def load_dataset(path):
    num_img = 1000
    with gzip.open(path, 'rb') as infile:
        data = np.frombuffer(infile.read(), dtype=np.uint8).reshape(num_img, 784)
    return data

def send_result(pred, testset_id,latency):
    url = 'https://courses.engr.illinois.edu/ece498icc/sp2019/lab2_request_dataset.php'
    values = {'request': 'verify', 
              'netid':'smeng10', 
              'team':'msp', 
              'testset_id' : testset_id, 
              'prediction' : pred, 
              'latency': latency}
    r = requests.post(url, data=values, allow_redirects=True)
    return r

def create_keras_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=3, kernel_size=5, padding='valid', activation='relu', input_shape=(28,28,1))) 
    model.add(tf.keras.layers.MaxPooling2D(strides=2))
    model.add(tf.keras.layers.Conv2D(filters=3, kernel_size=3, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(strides=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(50, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    model.summary()
    return model

def keras_test(ckpt, test_img):
    #KERAS checkpoint
    if ckpt == "FashionNet/":
        model = create_keras_model()
    elif ckpt == "FashionNetModified/":
        model = create_keras_model_modified()
    elif ckpt == "FashionNetMobile/":
        model = create_keras_model_mobilnet()
    else: raise KeyError
    model.load_weights(ckpt)
    
    start = time.time()
    pred = model.predict(test_img)
    end = time.time()
    
    pred = ''.join(list(map(lambda x: str(x.argmax()),pred)))
    return pred,calc_latency(start,end)

def autotest(ckpt = "all"):
    if ckpt == "all":
        autotest(ckpt = "FashionNet/")
        autotest(ckpt = "FashionNetModified/")
        autotest(ckpt = "FashionNetMobile/")
        return
    data = get_testset()
    imgs = data[0].reshape((data[0].shape[0], 28, 28, 1))/255.0
    #test keras
    pred, latency = keras_test(ckpt,imgs)
    r = send_result(pred=pred, testset_id=data[1], latency=latency)
    acc = int(r.text)/1000
    Score = acc / (5 * latency)
    print(f'checkpoint {ckpt} score is {Score}, its accuracy is {acc}, latency is {latency}')
    
def calc_latency(start,end):
    return end-start

# 1.1 Latency evaluation of your FashionNet in lab2 running on Raspberry Pi. 

In [4]:
#r = autotest(ckpt="FashionNet/")

# 1.2 Implement a modified FashionNet

In [5]:
def create_keras_model_modified():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=3, kernel_size=5, padding='valid', activation='relu', input_shape=(28,28,1))) 
    model.add(tf.keras.layers.MaxPooling2D(strides=2))
    model.add(tf.keras.layers.DepthwiseConv2D(kernel_size=3, padding='same', activation='relu'))
    model.add(tf.keras.layers.SeparableConv2D(filters=3, kernel_size=1, padding='valid', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(strides=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(50, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    model.summary()
    return model

In [6]:
#new_model = create_keras_model_modified()

In [7]:
#history = new_model.fit(train_images, train_labels, validation_split=0.1, batch_size=64, epochs=10)

In [8]:
#new_model.save_weights(filepath='FashionNetModified/')

In [9]:
#autotest(ckpt="FashionNetModified/")

# 1.3 Explore your own design

In [10]:
def create_keras_model_mobilnet():
    model = keras.Sequential([
    keras.layers.Conv2D(3, (3, 3), strides=(1, 1), padding='same', activation=tf.nn.relu6, input_shape=(28, 28, 1)),
    
    keras.layers.DepthwiseConv2D((3, 3), depth_multiplier=1, strides=(1, 1), padding='same', activation=tf.nn.relu6),
    keras.layers.Conv2D(8, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6),
    keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-5),
    keras.layers.Dropout(0.1),
    
    keras.layers.DepthwiseConv2D((3, 3), depth_multiplier=1, strides=(2, 2), padding='same', activation=tf.nn.relu6),
    keras.layers.Conv2D(16, (1, 1), strides=(1, 1), padding='valid', activation=tf.nn.relu6),
    keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-5),
    keras.layers.Dropout(0.1),
    
    keras.layers.DepthwiseConv2D((3, 3), depth_multiplier=1, strides=(2, 2), padding='same', activation=tf.nn.relu6),
    keras.layers.Conv2D(32, (1, 1), strides=(1, 1), padding='valid', activation=tf.nn.relu6),
    keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-5),
    keras.layers.Dropout(0.1),
    
    keras.layers.DepthwiseConv2D((3, 3), depth_multiplier=1, strides=(2, 2), padding='same', activation=tf.nn.relu6),
    keras.layers.Conv2D(16, (1, 1), strides=(1, 1), padding='valid', activation=tf.nn.relu6),
    keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-5),
    keras.layers.Dropout(0.1),
    
#     keras.layers.DepthwiseConv2D((3, 3), depth_multiplier=1, strides=(2, 2), padding='same', activation=tf.nn.relu6),
#     keras.layers.Conv2D(8, (1, 1), strides=(1, 1), padding='valid', activation=tf.nn.relu6),
#     keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-5),
#     keras.layers.Dropout(0.1),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu6),
    keras.layers.Dense(64, activation=tf.nn.relu6),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.summary()
    return model

In [11]:
# new_model = create_keras_model_mobilnet()
# new_model.compile(optimizer='adam', 
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

In [12]:
# history = new_model.fit(train_images, train_labels, validation_split=0.1, batch_size=64, epochs=10)

In [13]:
# new_model.save_weights(filepath='FashionNetMobile/')

In [14]:
# autotest(ckpt='FashionNetMobile/')

In [18]:
autotest()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 24, 24, 3)         78        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 3)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 12, 12, 3)         84        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 3)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 108)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10900     
_________________________________________________________________
dense_13 (Dense)             (None, 50)                5050      
__________

NameError: name 'create_keras_model_modified' is not defined